In [0]:
!pip install pymorphy2
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [0]:
PATH_TO_DATA = '../ru_kw_eval_datasets-master/data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

On my local machine open file and save it as .csv. Unfortunately keywords lists were converted to the sring format, therefore in later using I decided to split them by word (not n-gram) and also normalize. 

In [0]:
from google.colab import files
uploaded = files.upload()

Saving data_text.csv to data_text.csv


In [0]:
data = pd.read_csv('data_text.csv')

In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [0]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

import re

def normalize_kw(text):
  
    words = re.findall('[а-яА-ЯЁё]+',text)
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [0]:
data['norm_content'] = data.content.apply(normalize)
data['norm_kw'] = data.keywords.apply(normalize_kw)

In [0]:
data.head()

,Unnamed: 0,content,keywords,summary,title,url,norm_content,norm_kw
0,0,"Многие интересуются, зачем нужна «Яблоку» моло...","['яблоко', 'молодежь', 'молодежное яблоко']",NaN,"""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,многие интересоваться нужный яблоко молодёжный...,"[яблоко, молодёжь, молодёжный, яблоко]"
1,1,Вчера «Газпром» снизил верхнюю планку прогноза...,"['газпром', 'газ']",NaN,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,вчера газпром снизить верхний планка прогноз с...,"[газпром, газ]"
2,2,Долголетний труд Евгения Витковского на ниве п...,"['франсуа рабле', 'сервантес', 'шекспир', 'кон...","Евгений Витковский о том, как Босх протягивает...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,долголетний труд евгений витковский нива перев...,"[франсуа, рабле, сервантес, шекспир, конана, д..."
3,3,В Ленинском районном суде продолжаются слушани...,"['владивосток', 'суд', 'ким', 'футина', 'выбор...",Фигурантке уголовного дела о фальсификации выб...,"Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,ленинский районный суд продолжаться слушание д...,"[владивосток, суд, ким, футиный, выбор, боевой..."
4,4,В 2012 году российская столица резко увеличила...,"['новая москва', 'подмосковье', 'благоустройст...",Лучшие проекты благоустройства общественных пр...,Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,2012 год российский столица резко увеличиться ...,"[новый, москва, подмосковье, благоустройство, ..."


Variant #1. Use Rake without/with content normalization and keywords normalization.

In [0]:
!pip install rake_nltk

In [0]:
from rake_nltk import Metric, Rake

In [0]:
def rank(text):
    # To use it with a specific language supported by nltk.
    r = Rake(language='ru', stopwords=stops,
    punctuations=punct, ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO, min_length=1, max_length=3)
    r.extract_keywords_from_text(text)
    kw = r.get_ranked_phrases()
    return normalize(' '.join(kw)).split(' ')

In [0]:
data['pred_kw_CONTENT'] = data.content.apply(rank)

In [0]:
evaluate(data['norm_kw'], data['pred_kw_CONTENT'])

Precision -  0.03
Recall -  0.72
F1 -  0.05
Jaccard -  0.03


In [0]:
data['pred_kw_NORM_CONTENT'] = data.norm_content.apply(rank)
evaluate(data['norm_kw'], data['pred_kw_NORM_CONTENT'])

Precision -  0.04
Recall -  0.04
F1 -  0.03
Jaccard -  0.02


As the results show, the RAKE works better with unnormalized texts.

In [0]:
data.head()

,Unnamed: 0,content,keywords,summary,title,url,norm_content,norm_kw,pred_kw_CONTENT,pred_kw_NORM_CONTENT
0,0,"Многие интересуются, зачем нужна «Яблоку» моло...","['яблоко', 'молодежь', 'молодежное яблоко']",NaN,"""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,многие интересоваться нужный яблоко молодёжный...,"[яблоко, молодёжь, молодёжный, яблоко]","[терять, целый, год, существовать, высокий, за...",[]
1,1,Вчера «Газпром» снизил верхнюю планку прогноза...,"['газпром', 'газ']",NaN,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,вчера газпром снизить верхний планка прогноз с...,"[газпром, газ]","[это, следовать, добавить, шельф, баренцев, мо...","[вр, новатэк]"
2,2,Долголетний труд Евгения Витковского на ниве п...,"['франсуа рабле', 'сервантес', 'шекспир', 'кон...","Евгений Витковский о том, как Босх протягивает...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,долголетний труд евгений витковский нива перев...,"[франсуа, рабле, сервантес, шекспир, конана, д...","[являться, главное, редактор, часто, бессмысле...","[гильдия, похоже, говорить, 4, месяц, лично, р..."
3,3,В Ленинском районном суде продолжаются слушани...,"['владивосток', 'суд', 'ким', 'футина', 'выбор...",Фигурантке уголовного дела о фальсификации выб...,"Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,ленинский районный суд продолжаться слушание д...,"[владивосток, суд, ким, футиный, выбор, боевой...","[хотя, прежде, защитник, фальсификация, избира...","[летие, мчс, многие]"
4,4,В 2012 году российская столица резко увеличила...,"['новая москва', 'подмосковье', 'благоустройст...",Лучшие проекты благоустройства общественных пр...,Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,2012 год российский столица резко увеличиться ...,"[новый, москва, подмосковье, благоустройство, ...","[фон, такой, гигант, установить, водоохранный,...","[гигант, спортивно]"


Variant #2. Package  YAKE (https://pypi.org/project/yake/)

In [0]:
!pip install yake

In [0]:
import yake

In [0]:
import numpy as np
def yake_extractor(text):
  simple_kwextractor = yake.KeywordExtractor(lan="ru", n=1, dedupLim=0.8, windowsSize=2, top=20)
  scores = simple_kwextractor.extract_keywords(text)
  return [i for j,i in scores]

data['yake_CONTENT'] = data.content.apply(yake_extractor)

In [0]:
evaluate(data.norm_kw, data.yake_CONTENT)

Precision -  0.05
Recall -  0.18
F1 -  0.08
Jaccard -  0.04


In [0]:
data['yake_CONTENT_NORM'] = data.norm_content.apply(yake_extractor)

In [0]:
evaluate(data.norm_kw, data.yake_CONTENT_NORM)

Precision -  0.14
Recall -  0.44
F1 -  0.2
Jaccard -  0.12


Yake KW extractor works better witn normalized texts and shows a great result. I use it with some parameters, which are proposed by the authors of model on  the package home page.

In [0]:
def yake_extractor_tag(text):
  simple_kwextractor = yake.KeywordExtractor(lan="ru", n=1, dedupLim=0.8, windowsSize=2, top=20)
  scores = simple_kwextractor.extract_keywords(text)
  words = [i for j,i in scores]
  words = [morph.parse(word)[0].normal_form for word in words if morph.parse(word)[0].tag.POS in ["NOUN", 'ADJF', 'ADJS']]
  
  return words

In [0]:
data['yake_CONTENT_NORM_TAG'] = data.norm_content.apply(yake_extractor_tag)

In [0]:
evaluate(data.norm_kw, data.yake_CONTENT_NORM_TAG)

Precision -  0.16
Recall -  0.43
F1 -  0.22
Jaccard -  0.13


This result is achived by using normalization for yake kw results, also output of model are filtred by tags. Only noun and adj didn't deleted.

In [0]:
def normalize_tag(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word and word not in stops]
    words = [morph.parse(word)[0].normal_form for word in words if morph.parse(word)[0].tag.POS in ["NOUN", 'ADJF', 'ADJS']]

    return ' '.join(words)

In [0]:
data['content_norm_tag'] = data.content.apply(normalize_tag)

In [0]:
data['yake_CONTENT_NORM_tagged_TAG'] = data.content_norm_tag.apply(yake_extractor_tag)

In [0]:
evaluate(data.norm_kw, data.yake_CONTENT_NORM_tagged_TAG)

Precision -  0.14
Recall -  0.46
F1 -  0.21
Jaccard -  0.12


If we use normalized texts, which contane only nouns and adjs for training yake, F1 of the result decreases. 

In [0]:
data.head()

,Unnamed: 0,content,keywords,summary,title,url,norm_content,norm_kw,pred_kw_CONTENT,pred_kw_NORM_CONTENT,yake_CONTENT,yake_CONTENT_NORM,yake_CONTENT_NORM_TAG,content_norm_tag,yake_CONTENT_NORM_tagged_TAG
0,0,"Многие интересуются, зачем нужна «Яблоку» моло...","['яблоко', 'молодежь', 'молодежное яблоко']",NaN,"""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,многие интересоваться нужный яблоко молодёжный...,"[яблоко, молодёжь, молодёжный, яблоко]","[терять, целый, год, существовать, высокий, за...",[],"[яблоко, молодежное, акции, россии, фракция, н...","[яблоко, молодёжный, акция, активист, проводит...","[яблоко, молодёжный, акция, активист, московск...",нужный яблоко молодёжный фракция основной зада...,"[яблоко, молодёжный, акция, активист, московск..."
1,1,Вчера «Газпром» снизил верхнюю планку прогноза...,"['газпром', 'газ']",NaN,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,вчера газпром снизить верхний планка прогноз с...,"[газпром, газ]","[это, следовать, добавить, шельф, баренцев, мо...","[вр, новатэк]","[куб, млрд., газа, газпром, россии, куб., стра...","[куб, миллиард, метр, газа, добыча, газпром, н...","[куб, миллиард, метр, газа, добыча, газпром, н...",газпром верхний планка прогноз собственный доб...,"[метр, миллиард, куб, газа, газпром, добыча, д..."
2,2,Долголетний труд Евгения Витковского на ниве п...,"['франсуа рабле', 'сервантес', 'шекспир', 'кон...","Евгений Витковский о том, как Босх протягивает...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,долголетний труд евгений витковский нива перев...,"[франсуа, рабле, сервантес, шекспир, конана, д...","[являться, главное, редактор, часто, бессмысле...","[гильдия, похоже, говорить, 4, месяц, лично, р...","[евгения, витковского, поэзии, москвы, роман, ...","[книга, роман, писать, век, поэзия, выйти, стр...","[книга, роман, поэзия, страница, стихотворение...",долголетний труд евгений витковский нива перев...,"[книга, роман, мир, поэзия, москва, страница, ..."
3,3,В Ленинском районном суде продолжаются слушани...,"['владивосток', 'суд', 'ким', 'футина', 'выбор...",Фигурантке уголовного дела о фальсификации выб...,"Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,ленинский районный суд продолжаться слушание д...,"[владивосток, суд, ким, футиный, выбор, боевой...","[хотя, прежде, защитник, фальсификация, избира...","[летие, мчс, многие]","[ким, ленинском, думы, владивостока, выборах, ...","[ким, зинаида, футиный, рубль, видео, журналис...","[ким, зинаида, футиный, рубль, видео, журналис...",ленинский районный суд слушание дело экс-депут...,"[ким, зинаида, видео, журналист, футиный, рубл..."
4,4,В 2012 году российская столица резко увеличила...,"['новая москва', 'подмосковье', 'благоустройст...",Лучшие проекты благоустройства общественных пр...,Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,2012 год российский столица резко увеличиться ...,"[новый, москва, подмосковье, благоустройство, ...","[фон, такой, гигант, установить, водоохранный,...","[гигант, спортивно]","[москвы, новой, московской, тинао, парк, площа...","[новый, москва, площадь, территория, парк, раз...","[новый, москва, площадь, территория, парк, раз...",год российский столица размер результат присое...,"[новый, площадь, москва, территория, развитие,..."


Variant #3. TF-IDF

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['norm_content'])
texts_vectors = tfidf.transform(data['norm_content'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
print(len(keywords))
data['title_norm'] = data.title.apply(normalize_tag)

999


In [0]:
data['kw_tfidf_title'] = keywords
data['kw_tfidf_title'] = data['kw_tfidf_title']+data['title_norm'].apply(lambda x: x.split())

In [0]:
data.kw_tfidf_title.head()

0    [яблоко, молодёжный, активист, акция, дарья, д...
1    [миллиард куб, куб метр, куб, газпром, газа, м...
2    [роман, книга, писать, жанр, издать, стихотвор...
3    [ким, зинаида, видео, экспертиза, свидетель, л...
4    [га, площадь, парковый, парка, парк, территори...
Name: kw_tfidf_title, dtype: object

In [0]:
def clear_kw(words):
  #for word in words:
    #print(morph.parse(word)[0].tag.POS)
  words = [morph.parse(word)[0].normal_form for word in words if morph.parse(word)[0].tag.POS in ["NOUN", 'ADJF', 'ADJS']]
  return words
data.kw_tfidf_title = data.kw_tfidf_title.apply(clear_kw)

In [0]:
evaluate(data.norm_kw, data.kw_tfidf_title)

Precision -  0.2
Recall -  0.39
F1 -  0.25
Jaccard -  0.15


Usage of tf-idf  kws with normalized title (filtered also by tags) can shows the above-presented result.

Variant #4. Embeddings

In [0]:
text_data = ''
for i in data.content.tolist():
    text_data+=i
    text_data+=' '
text_data = text_data.replace('\n', '. ')
text_data = text_data.replace('.. ', '. ')

In [0]:
def norm_for_emb(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [0]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

sent_tokenize_list = sent_tokenize(text_data)
clear_sents=[]
for i in sent_tokenize_list:
    clear_sent = []
    clear_sent = norm_for_emb(i)
    clear_sents.append(clear_sent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!pip install gensim

In [0]:
import gensim
simple_model = gensim.models.FastText(clear_sents, size=300, window=5,
                                      min_count=1, workers=2)

In [0]:
def yake_extractor_tag_80(text):
  simple_kwextractor = yake.KeywordExtractor(lan="ru", n=1, dedupLim=0.8, windowsSize=2, top=80)
  scores = simple_kwextractor.extract_keywords(text)
  words = [i for j,i in scores]
  words = [morph.parse(word)[0].normal_form for word in words if morph.parse(word)[0].tag.POS in ["NOUN", 'ADJF', 'ADJS']]
  return words

data['yake_CONTENT_NORM_TAG_80'] = data.norm_content.apply(yake_extractor_tag_80)

Extract first 80 unigrams from normalised texts. The result words are also normalized and filterd by tags.

In [0]:
sets = list(zip(data.kw_tfidf_title.tolist(), data.yake_CONTENT_NORM_TAG_80.tolist()))
sets = [list(set(i).intersection(set(j))) for i, j in sets ]
data['intesection_tf_yark'] = pd.Series(sets)
evaluate(data.norm_kw, data['intesection_tf_yark'])

Precision -  0.24
Recall -  0.36
F1 -  0.27
Jaccard -  0.17


Intersection of key words from tf-idf+title and Yake - 80 can make our model better.

In [0]:
from scipy.spatial.distance import cosine
#example = data.yake_CONTENT_NORM_TAG_80[0]
def smth(example):
  word2ind = {word : simple_model[word] for word in example}
  ind2word = {tuple(i):j for j,i in word2ind.items()}
  vecs = [simple_model[word] for word in example]
  mean_dists = []
  median_dists = []

  for in1,vec1 in enumerate(vecs):
      distances = []
      for in2,vec2 in enumerate(vecs):
        if in1!=in2:
          distances.append((ind2word[tuple(vec2)],cosine(vec1,vec2)))
      distances = sorted(distances, key=lambda tup: tup[1])

      #print(ind2word[tuple(vec1)], distances)
      #print('################################################')
      mean_dists.append((ind2word[tuple(vec1)], np.mean(np.array([i for j, i in distances])), distances))
      median_dists.append((ind2word[tuple(vec1)], np.median(np.array([i for j, i in distances])), distances ))

  sorted_mean_dists = sorted(mean_dists, key=lambda tup: tup[1])
  sorted_median_dists = sorted(median_dists, key=lambda tup: tup[1])

  s = 0
  resss = []
  while s<=15 and s<len(sorted_median_dists):
    info = sorted_median_dists[s]
    #print(info)
    f = 0 
    while f<=5:
      res=[]
      #print(info[2][f][0])
      res.append(info[2][f][0])
      f+=1
    resss.extend(res)
    s+=1
  return list(set(resss))

data['emb_kw'] = data.yake_CONTENT_NORM_TAG_80.apply(smth)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
evaluate(data.norm_kw, data.emb_kw)

Precision -  0.09
Recall -  0.16
F1 -  0.11
Jaccard -  0.06


Using yake 80 key words unigrams I tried to find some clasters according to the cosine  simularity and get more central terms from them. But it doesn't work.

In [0]:
data['intesection_tf_yark_emb'] = data.intesection_tf_yark+data.emb_kw
evaluate(data.norm_kw, data['intesection_tf_yark_emb'])

Precision -  0.14
Recall -  0.42
F1 -  0.2
Jaccard -  0.12


Intersection of tf-idf, yake-80 and embeddings doesn't increase the presison of our model.

In [0]:
data = data.drop(['content_split', 'content_vec'], axis = 1)
data.head()

,Unnamed: 0,content,keywords,summary,title,url,norm_content,norm_kw,pred_kw_CONTENT,pred_kw_NORM_CONTENT,yake_CONTENT,yake_CONTENT_NORM,yake_CONTENT_NORM_TAG,content_norm_tag,yake_CONTENT_NORM_tagged_TAG,title_norm,kw_tfidf_title,yake_CONTENT_NORM_TAG_80
0,0,"Многие интересуются, зачем нужна «Яблоку» моло...","['яблоко', 'молодежь', 'молодежное яблоко']",NaN,"""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,многие интересоваться нужный яблоко молодёжный...,"[яблоко, молодёжь, молодёжный, яблоко]","[терять, целый, год, существовать, высокий, за...",[],"[яблоко, молодежное, акции, россии, фракция, н...","[яблоко, молодёжный, акция, активист, проводит...","[яблоко, молодёжный, акция, активист, московск...",нужный яблоко молодёжный фракция основной зада...,"[яблоко, молодёжный, акция, активист, московск...",молодёжный яблоко оппозиционный деятельность о...,"[яблоко, молодёжный, активист, акция, дарья, д...","[яблоко, молодёжный, акция, активист, московск..."
1,1,Вчера «Газпром» снизил верхнюю планку прогноза...,"['газпром', 'газ']",NaN,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,вчера газпром снизить верхний планка прогноз с...,"[газпром, газ]","[это, следовать, добавить, шельф, баренцев, мо...","[вр, новатэк]","[куб, млрд., газа, газпром, россии, куб., стра...","[куб, миллиард, метр, газа, добыча, газпром, н...","[куб, миллиард, метр, газа, добыча, газпром, н...",газпром верхний планка прогноз собственный доб...,"[метр, миллиард, куб, газа, газпром, добыча, д...",газпром,"[миллиард куб, куб метр, куб, газпром, газа, м...","[куб, миллиард, метр, газа, добыча, газпром, н..."
2,2,Долголетний труд Евгения Витковского на ниве п...,"['франсуа рабле', 'сервантес', 'шекспир', 'кон...","Евгений Витковский о том, как Босх протягивает...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,долголетний труд евгений витковский нива перев...,"[франсуа, рабле, сервантес, шекспир, конана, д...","[являться, главное, редактор, часто, бессмысле...","[гильдия, похоже, говорить, 4, месяц, лично, р...","[евгения, витковского, поэзии, москвы, роман, ...","[книга, роман, писать, век, поэзия, выйти, стр...","[книга, роман, поэзия, страница, стихотворение...",долголетний труд евгений витковский нива перев...,"[книга, роман, мир, поэзия, москва, страница, ...",бесконечный партия четырехмерный шахматы,"[роман, книга, жанр, стихотворение, перевод, ч...","[книга, роман, поэзия, страница, стихотворение..."
3,3,В Ленинском районном суде продолжаются слушани...,"['владивосток', 'суд', 'ким', 'футина', 'выбор...",Фигурантке уголовного дела о фальсификации выб...,"Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,ленинский районный суд продолжаться слушание д...,"[владивосток, суд, ким, футиный, выбор, боевой...","[хотя, прежде, защитник, фальсификация, избира...","[летие, мчс, многие]","[ким, ленинском, думы, владивостока, выборах, ...","[ким, зинаида, футиный, рубль, видео, журналис...","[ким, зинаида, футиный, рубль, видео, журналис...",ленинский районный суд слушание дело экс-депут...,"[ким, зинаида, видео, журналист, футиный, рубл...",экс-депутат фальсификация выбор член боевой бр...,"[ким, зинаида, видео, экспертиза, свидетель, л...","[ким, зинаида, футиный, рубль, видео, журналис..."
4,4,В 2012 году российская столица резко увеличила...,"['новая москва', 'подмосковье', 'благоустройст...",Лучшие проекты благоустройства общественных пр...,Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,2012 год российский столица резко увеличиться ...,"[новый, москва, подмосковье, благоустройство, ...","[фон, такой, гигант, установить, водоохранный,...","[гигант, спортивно]","[москвы, новой, московской, тинао, парк, площа...","[новый, москва, площадь, территория, парк, раз...","[новый, москва, площадь, территория, парк, раз...",год российский